In [ ]:
-- create database
CREATE SCHEMA tech_electro;

-- Data Exploration
SELECT * FROM external_factors;
SELECT * FROM product_information;
SELECT * FROM sales_data;

--  understanding the structure of the data
DESCRIBE external_factors;
DESCRIBE product_information;
DESCRIBE sales_data;

-- Data Cleaning: Data Type
-- external factor

ALTER TABLE external_factors
ADD COLUMN new_sales_date DATE;

SET sql_safe_update = o;

UPDATE external_factors
set new_sales_date = STR_TO_DATE(sales_date, "%d/%m/%y");

ALTER TABLE external_factors
DROP sales_date;

ALTER TABLE external_factors
CHANGE COLUMN new_sales_date sales_date DATE;

ALTER TABLE external_factors
MODIFY COLUMNS GDP DECIMAL(15, 2);

ALTER TABLE external_factors
MODIFY COLUMNS Inflation_Rate DECIMAL(5, 2);

ALTER TABLE external_factors
MODIFY COLUMNS Seasonal_Factor DECIMAL(5, 2);

-- product data
ALTER TABLE product_information
ADD COLUMN new_promotion ENUM ("yes", "no");

UPDATE product_information
SET new_promotion = CASE
	WHEN promotions = "yes" THEN "yes"
    WHEN promotions = "no" THEN "no"
    ELSE null
END;

ALTER TABLE product_information
DROP promotions;

ALTER TABLE product_information
CHANGE COLUMN new_promotion promotions ENUM("yes", "no");

-- sales data
ALTER TABLE sales_data
ADD COLUMN new_sales_date DATE;

UPDATE sales_data
set new_sales_date = STR_TO_DATE(sales_date, "%d/%m/%y");

ALTER TABLE sales_data
DROP sales_date;

ALTER TABLE sales_data
CHANGE COLUMN new_sales_date sales_date DATE;

-- Data cleaning ; Missing data
-- External factors
SELECT
	SUM(CASE WHEN sales_date IS NULL THEN 1 ELSE 0 END)  AS missing_sales_date,
    SUM(CASE WHEN GDP IS NULL THEN 1 ELSE 0 END)  AS missing_GDP,
    SUM(CASE WHEN Inflation_Rate IS NULL THEN 1 ELSE 0 END)  AS missing_inflation,
    SUM(CASE WHEN Seasonal_Factor IS NULL THEN 1 ELSE 0 END)  AS missing_seasonal_factor
FROM
	external_factors;

-- product information
SELECT
	SUM(CASE WHEN Product_ID IS NULL THEN 1 ELSE 0 END)  AS missing_product_id,
    SUM(CASE WHEN Product_Category IS NULL THEN 1 ELSE 0 END)  AS missing_product_category,
    SUM(CASE WHEN promotion IS NULL THEN 1 ELSE 0 END)  AS missing_promotion
FROM
	product_information;

-- sales_date
SELECT
	SUM(CASE WHEN sales_date IS NULL THEN 1 ELSE 0 END)  AS missing_sales_date,
    SUM(CASE WHEN Product_ID IS NULL THEN 1 ELSE 0 END)  AS missing_product_id,
    SUM(CASE WHEN Product_Cost IS NULL THEN 1 ELSE 0 END)  AS missing_product_cost,
    SUM(CASE WHEN Inventory_Quantity IS NULL THEN 1 ELSE 0 END)  AS missing_inventory_quantity
FROM
	sales_data;

-- data cleaning; Duplicate values
-- external factors

SELECT
	sales_date,
    COUNT(*) AS count
FROM external_factors
GROUP BY sales_date
HAVING count > 1;

SELECT
	COUNT(*)
FROM(
	SELECT
		sales_date,
		COUNT(*) AS count
	FROM external_factors
	GROUP BY sales_date
	HAVING count > 1) AS dup;

-- product data
SELECT
	Product_ID,
    Product_Category,
    COUNT(*) AS count
FROM product_information
GROUP BY Product_ID
HAVING count > 1;

SELECT
	COUNT(*)
FROM(
	SELECT
		Product_ID,
		Product_Category,
		COUNT(*) AS count
	FROM product_information
	GROUP BY Product_ID
	HAVING count > 1) as dup;

-- sales data
SELECT
		Product_ID,
		sales_date,
		COUNT(*) AS count
	FROM sales_data
	GROUP BY Product_ID, sales_date
	HAVING count > 1;

-- data cleaning; Handling Duplicate values
-- external factors

DELETE e1 from external_factors e1
INNER JOIN(
	SELECT
		sales_date,
		ROW_NUMBER() OVER(PARTITION BY sales_date ORDER BY sales_date) AS RN
	FROM external_factors) e2
ON e1.sales_date = e2.sales_date
WHERE e2.RN > 1;

-- product data
DELETE p1 from product_information p1
INNER JOIN(
	SELECT
		Product_ID,
		ROW_NUMBER() OVER(PARTITION BY Product_ID ORDER BY Product_ID) AS PN
	FROM product_information ) p2
ON p1.Product_ID = p2.Product_ID
WHERE p2.PN > 1;

-- Data Integration
-- sales data and product data
CREATE VIEW sales_product_data As
SELECT
	s.Product_ID,
    s.Inventory_Quantity,
    s.Product_Cost,
    s.sales_date,
    p.Product_Category,
    p.promotions
FROM sales_data s
JOIN product_information p
ON s.Product_ID = p.Product_ID;

-- combine sales_product_data with external factor

CREATE VIEW inventory_data AS
SELECT
	sp.Product_ID,
    sp.Inventory_Quantity,
    sp.Product_Cost,
    sp.sales_date,
    sp.Product_Category,
    sp.promotions,
    e.GDP,
    e.Inflation_Rate,
    e.Seasonal_factor
FROM sales_product_data sp
LEFT JOIN external_factors e
ON sp.sales_date = e.external_factor;

-- Descriptive Analysis
SELECT
	Product_ID,
    AVG(Inventory_Quantity * Product_Cost) AS avg_sales
FROM inventory_data
GROUP BY Product_ID
ORDER By avg_sales DESC;

-- median stock level
SELECT
	Product_ID,
	AVG(Inventory_Quantity) AS median_stock
FROM(
	SELECT
		Product_ID,
        Inventory_QUantity,
        ROW_NUMBER() OVER(PARTITION BY Product_ID ORDER BY Inventory_Quantity) AS row_num_asc,
        ROW_NUMBER() OVER(PARTITION BY Product_ID ORDER BY Inventory_Quantity DESC) AS row_num_desc
	FROM inventory_data) AS sub_query
WHERE row_num_asc IN(row_num_desc, row_num_desc -1, row_num_desc + 1)
GROUP BY Product_ID;

-- product performance metrics
SELECT
	Product_ID,
	ROUND(SUM(Inventory_Quantity * Product_Cost)) AS total_sales
FROM inventory_data
GROUP BY Product_ID
ORDER BY total_sales DESC;

-- identify high demand products based on average sales
WITH high_demand_product AS(
	SELECT
		Product_ID,
        AVG(Inventory_Quantity) AS avg_sales
	FROM inventory_data
    GROUP BY Product_ID
    HAVING avg_sales >(
		SELECT
			AVG(Inventory_Quantity) * 0.95
		FROM sales_data
        )
	)
-- calculate stock out frequency for high demand products
SELECT
	s.Product_ID,
    COUNT(*) AS stock_out_frequency
FROM inventory_data s
WHERE s.Product_ID IN (select Product_ID FROM high_demand_product)
AND s.Inventory_Quantity =0
GROUP BY s.Product_ID;

-- influenece of external factor
SELECT
	Product_ID,
    AVG(CASE WHEN 'GDP' > 0 THEN Inventory_Quantity ELSE NULL END) AS avg_sales_positive_gdp,
	AVG(CASE WHEN 'GDP' <= 0 THEN Inventory_Quantity ELSE NULL END) AS avg_sales_negative_gdp
FROM inventory_data
GROUP BY Product_ID
HAVING avg_sales_positive_gdp IS NOT NULL;

-- inflation rate
SELECT
	Product_ID,
    AVG(CASE WHEN 'Inflation_Rate' > 0 THEN Inventory_Quantity ELSE NULL END) AS avg_sales_positive_inflation,
	AVG(CASE WHEN 'Inflation_Rate' <= 0 THEN Inventory_Quantity ELSE NULL END) AS avg_sales_negative_inflation
FROM inventory_data
GROUP BY Product_ID
HAVING avg_sales_positive_inflation IS NOT NULL;

--  Inventory optimization

WITH inventory_calculation AS (
    SELECT
        Product_ID,
        AVG(rolling_avg_sales) AS avg_rolling_sales,
        AVG(rolling_variance) AS avg_rolling_variance
    FROM (
        SELECT
            Product_ID,
            sales_date,
            AVG(daily_sales) OVER (PARTITION BY Product_ID ORDER BY sales_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_avg_sales,
            AVG(squared_diff) OVER (PARTITION BY Product_ID ORDER BY sales_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_variance
        FROM (
            SELECT
                Product_ID,
                sales_date,
                Inventory_Quantity * Product_Cost AS daily_sales,
                power(Inventory_Quantity * Product_Cost -
                      AVG(Inventory_Quantity * Product_Cost) OVER (PARTITION BY Product_ID ORDER BY sales_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2)
                AS squared_diff
            FROM Inventory_data
        ) subquery1
    ) subquery2
    GROUP BY Product_ID
)
SELECT
    Product_ID,
    avg_rolling_sales * 7 AS lead_time_demand,
    1.645 *(avg_rolling_sales * 7)  AS safety_stock,
    (avg_rolling_sales * 7) * (1.645 *(avg_rolling_sales * 7)) AS reorder_point
FROM inventory_calculation;

-- Create inventory optimization table
CREATE TABLE inventory_optimization(
	Product_ID INT,
    Reorder_Point DOUBLE
);

-- create stored procedure to recalculate reorder point
DELIMITER //

CREATE PROCEDURE recalculated_reorder_points(IN ProductID INT)
BEGIN
    DECLARE Reorder_Point DOUBLE;
    DECLARE Lead_Time_Demand DOUBLE;
    DECLARE Avg_Rolling_Sales DOUBLE;
    DECLARE Avg_Rolling_Variance DOUBLE;
    DECLARE Safety_Stock DOUBLE;

    -- Fetch average rolling sales and variance for the given Product_ID
    SELECT
        AVG(rolling_avg_sales),
        AVG(rolling_variance)
    INTO Avg_Rolling_Sales, Avg_Rolling_Variance
    FROM (
        SELECT
            Product_ID,
            sales_date,
            AVG(daily_sales) OVER (PARTITION BY Product_ID ORDER BY sales_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_avg_sales,
            AVG(squared_diff) OVER (PARTITION BY Product_ID ORDER BY sales_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS rolling_variance
        FROM (
            SELECT
                Product_ID,
                sales_date,
                Inventory_Quantity * Product_Cost AS daily_sales,
                POWER(Inventory_Quantity * Product_Cost -
                      AVG(Inventory_Quantity * Product_Cost) OVER (PARTITION BY Product_ID ORDER BY sales_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2)
                AS squared_diff
            FROM Inventory_data
        ) inner_derived
        WHERE Product_ID = ProductID
    ) outer_derived;

    -- Calculate Lead Time Demand (assuming lead time = 7 days)
    SET Lead_Time_Demand = Avg_Rolling_Sales * 7;

    -- Calculate Safety Stock (Z-score = 1.645 for 95% confidence)
    SET Safety_Stock = 1.645 * SQRT(Avg_Rolling_Variance);

    -- Calculate Reorder Point
    SET Reorder_Point = Lead_Time_Demand + Safety_Stock;

    -- Return the calculated values
    SELECT
        ProductID AS Product_ID,
        Lead_Time_Demand AS lead_time_demand,
        Safety_Stock AS safety_stock,
        Reorder_Point AS reorder_point;
END //

DELIMITER ;

-- make inventory data into a table
CREATE TABLE inventory_table AS SELECT * FROM inventory_data;

-- create the triggers
DELIMITER



-- overstock and understock situation
WITH rolling_sales AS (
    SELECT
        Product_ID,
        sales_date,
        AVG(Inventory_Quantity * Product_Cost) OVER (
            PARTITION BY Product_ID
            ORDER BY sales_date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_avg_sales
    FROM inventory_table
),

stockoutdays AS (
    SELECT
        Product_ID,
        COUNT(*) AS stockout_days
    FROM inventory_table
    WHERE Inventory_Quantity = 0
    GROUP BY Product_ID
)

SELECT
    f.Product_ID,
    AVG(f.Inventory_Quantity * f.Product_Cost) AS inventory_value,
    AVG(rs.rolling_avg_sales) AS avg_rolling_sales,
    COALESCE(sd.stockout_days, 0) AS stockout_days
FROM inventory_table f
JOIN rolling_sales rs
    ON f.Product_ID = rs.Product_ID
    AND f.sales_date = rs.sales_date
LEFT JOIN stockoutdays sd
    ON f.Product_ID = sd.Product_ID
GROUP BY f.Product_ID,sd.stockout_days;

-- monitor and adjust
DELIMITER //
CREATE PROCEDURE monitor_inventory_level()
BEGIN
SELECT
	Product_ID,
    AVG(Inventory_Quantity) AS avg_inventory
FROM inventory_table
GROUP BY Product_ID
ORDER BY avg_inventory DESC;
END //
DELIMITER ;

-- monitor sales trend
DELIMITER //

CREATE PROCEDURE monitor_sales_trend()
BEGIN
    SELECT
        Product_ID,
        sales_date,
        AVG(Inventory_Quantity * Product_Cost) OVER (
            PARTITION BY Product_ID
            ORDER BY sales_date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS rolling_avg_sales
    FROM inventory_table;
END //

DELIMITER ;

-- monitor stockout frequencies
DELIMITER //

CREATE PROCEDURE monitor_stockout_frequencies()
BEGIN
	SELECT
		Product_ID,
		COUNT(*) AS stockout_days
	FROM inventory_table
	Where inventory_Quantity = 0
	GROUP BY Product_ID
	ORDER BY stockout_days DESC;
END //

DELIMITER ;

-- feedback loop establishment
-- Feedback Portal: let's create another online platform where stakeholders can share feedbacks on inventory performance and challenges
-- Review Meetings: organize periodic sessions to discuss inventory performance and gather direct insights.
-- System Monitoring: use established SQL stored procedures to track system metrics with deviation from expected flagged for reviews

-- refinement based on feedback
-- Feedback Analysis: regularly compile and scrutinize feedback to identify recurring themes or pressing issues
-- Action Implementation: Prioritize and act on the feedback to adjust reorder points, safety stock levels and overall processes.
-- Change Communication: Inform stakeholders about changes,underscoring the value of their feedback and ensure transparency

-- Insights and Recommendations
-- General Insights

-- Inventory Discrepancies: The initial stages of the analysis reviewed discrepancies in the inventory level,
-- with the instances of both  overstockingand understocking. These inconsistences were contributing to capital inefficiencies and customer dissatisfaction

-- Sales Trend and External Influence: The analysis indicated that sales trend were notably influenced by various external factors
-- recognizing these patterns provides an opportunity to forecast demand more accurately.

-- Suboptimal Inventory Level: through the inventory optimization analysis, it was evident that the existing inventory level were not optomized for current sales trend
-- Products was identified that had either closed or excess inventory

-- Recommendations

-- Implement Dynamic Inventory Management: The company should transition from a static to a dynamic inventory system,
-- adjusting inventory level based on realtime sales trend, seasonality and external factors.

-- Optimize Reorder Points and Safety Stocks: utilize the reorder point and safety stocks calculated during the analysis to minimize stockout and reduce excess inventory
-- Regularly review those metrics to ensure they align with current market condition.

-- Enhancing Pricing Strategies: conduct a through review of product pricing strategies, especially for products identified as unprofitable
-- consider factors such as competitors pricing, market demand and product acquisition costs.alter

-- Reduce overstock: identified products that are constitenly overstocked and take steps to reduce their inventory level
-- This could include promotional sales, discounts or even discontinuing products with low sales performance.

-- Establish a feedback loop: develop a systematic approach to collect and analyze feedback from various stakeholders
-- use this feedback for continuous improvement and alignment with business objectives.

-- Regular monitoring and adjustment: adopt a proactive approach to inventory management by regularly monitoring key metrics
-- and making necessary adjustment to inventory levels, order quantity and safety stocks.

